1. Lib

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
from helper_statistic import *

Vix data

In [ ]:
vix_data = fetch_vix_data("2023-01-01", datetime.now().strftime("%Y-%m-%d"))
if vix_data is not None:
    vix_data.index = vix_data.index.tz_localize(None)

In [11]:
vix_data.tail()

Date
2025-12-10    15.77
2025-12-11    14.85
2025-12-12    15.74
2025-12-15    16.50
2025-12-16    16.48
Name: Close, dtype: float64

2. Strategy 1 - Analyst Day Volatility Arbitrage

In [4]:
def run_analyst_day_strategy(vix_data):
    ticker_symbol = "HOOD"
    event_date = "2024-12-04"
    
    event_dt = datetime.strptime(event_date, "%Y-%m-%d")
    entry_date = (event_dt - timedelta(days=7)).strftime("%Y-%m-%d") # T-5
    exit_date = (event_dt + timedelta(days=1)).strftime("%Y-%m-%d")  # T+1
    
    # Fetch Data
    data_start = (event_dt - timedelta(days=365)).strftime("%Y-%m-%d") # Need more history for GARCH
    data_end = (event_dt + timedelta(days=5)).strftime("%Y-%m-%d")
    hist = fetch_data(ticker_symbol, data_start, data_end)
    
    if hist.empty: return None

    try:
        entry_row = hist.asof(entry_date)
        exit_row = hist.asof(exit_date)
    except: return None
        
    spot_entry = entry_row['Close']
    strike = np.ceil(spot_entry) 
    
    # IV: GARCH + VIX Scaling
    # Calculate GARCH vol up to entry date
    hist_entry = hist.loc[:entry_date]
    garch_vol = get_garch_volatility(hist_entry)
    
    # VIX Scaling
    # If VIX is high, boost IV.
    # Baseline VIX ~ 20.
    current_vix = 20.0
    if vix_data is not None:
        try:
            current_vix = vix_data.asof(entry_date)
        except: pass
    
    vix_scalar = max(1.0, current_vix / 20.0)
    atm_vol = garch_vol * vix_scalar
    
    iv_entry = get_vol_skew(atm_vol, strike, spot_entry)
    
    expiry_date = "2024-12-06"
    expiry_dt = datetime.strptime(expiry_date, "%Y-%m-%d")
    entry_dt = datetime.strptime(entry_date, "%Y-%m-%d")
    dte_entry = (expiry_dt - entry_dt).days / 365.0
    
    # Binomial Pricing
    mid_price_entry = binomial_tree_price(spot_entry, strike, dte_entry, RISK_FREE_RATE, iv_entry, option_type='call')
    buy_price_entry, _ = apply_transaction_costs(mid_price_entry)
    
    # Exit
    spot_exit = exit_row['Close']
    exit_dt = datetime.strptime(exit_date, "%Y-%m-%d")
    dte_exit = (expiry_dt - exit_dt).days / 365.0
    
    iv_exit = iv_entry # Assume constant vol for simplicity (or crush)
    mid_price_exit = binomial_tree_price(spot_exit, strike, dte_exit, RISK_FREE_RATE, iv_exit, option_type='call')
    _, sell_price_exit = apply_transaction_costs(mid_price_exit)
    
    pnl = (sell_price_exit - buy_price_entry) / buy_price_entry
    
    print(f"Entry: ${buy_price_entry:.2f} (Mid: {mid_price_entry:.2f}, IV: {iv_entry:.2%})")
    print(f"Exit: ${sell_price_exit:.2f} (Mid: {mid_price_exit:.2f})")
    print(f"P&L: {pnl:.2%}")
    
    return {
        "Strategy": "Analyst Day",
        "Ticker": ticker_symbol,
        "Entry Date": entry_date,
        "Return": pnl
    }


3. Strategy 2 - Pre-Earnings Call Buying

In [21]:
def run_earnings_strategy(vix_data, tickers=['BLK', 'MSFT', 'NVDA', 'AAPL', 'AMD', 'TSLA', 'META', 'AMZN']):
    print("\n--- Strategy 2: Pre-Earnings Call Buying ---")
    results = []
    
    for ticker_symbol in tickers:
        try:
            ticker = yf.Ticker(ticker_symbol)
            earnings = ticker.get_earnings_dates(limit=12)
            if earnings is None or earnings.empty: continue
                
            now_aware = datetime.now().astimezone()
            past_earnings = earnings.index[earnings.index < now_aware]
            
            for event_dt in past_earnings[:2]:
                event_date = event_dt.strftime("%Y-%m-%d")
                entry_dt = event_dt - timedelta(days=4) # T-2
                entry_date = entry_dt.strftime("%Y-%m-%d")
                exit_dt = event_dt + timedelta(days=1) # T+1
                exit_date = exit_dt.strftime("%Y-%m-%d")
                
                data_start = (entry_dt - timedelta(days=365)).strftime("%Y-%m-%d")
                data_end = (exit_dt + timedelta(days=5)).strftime("%Y-%m-%d")
                hist = fetch_data(ticker_symbol, data_start, data_end)
                
                if hist.empty: continue
                hist.index = hist.index.tz_localize(None)
                
                try:
                    entry_row = hist.asof(entry_date)
                    exit_row = hist.asof(exit_date)
                except: continue
                    
                if pd.isna(entry_row['Close']) or pd.isna(exit_row['Close']): continue

                spot_entry = entry_row['Close']
                strike = np.ceil(spot_entry * 1.02) # 2% OTM
                
                # IV: GARCH + VIX + Earnings Premium
                hist_entry = hist.loc[:entry_date]
                garch_vol = get_garch_volatility(hist_entry)
                
                current_vix = 20.0
                if vix_data is not None:
                    try: current_vix = vix_data.asof(entry_date)
                    except: pass
                vix_scalar = max(1.0, current_vix / 20.0)
                
                atm_vol = garch_vol * vix_scalar * 1.5 # Earnings Premium
                iv_entry = get_vol_skew(atm_vol, strike, spot_entry)
                
                dte_entry = 10 / 365.0
                mid_price_entry = binomial_tree_price(spot_entry, strike, dte_entry, RISK_FREE_RATE, iv_entry, option_type='call')
                buy_price_entry, _ = apply_transaction_costs(mid_price_entry)
                
                # Exit
                spot_exit = exit_row['Close']
                dte_exit = (10 - 3) / 365.0
                
                # IV Crush
                iv_exit = (garch_vol * vix_scalar) * 1.0 # Crush back to normal (no premium)
                mid_price_exit = binomial_tree_price(spot_exit, strike, dte_exit, RISK_FREE_RATE, iv_exit, option_type='call')
                _, sell_price_exit = apply_transaction_costs(mid_price_exit)
                
                pnl = (sell_price_exit - buy_price_entry) / buy_price_entry
                
                results.append({
                    "Strategy": "Earnings",
                    "Ticker": ticker_symbol,
                    "Event Date": event_date,
                    "Return": pnl
                })
                print(f"{ticker_symbol} Earnings {event_date}: {pnl:.2%}")
                
        except Exception as e:
            print(f"Error {ticker_symbol}: {e}")
            
    return results

3. Strategy 3 - Asymmetric Alpha (Covered Call)

In [22]:
def run_covered_call_strategy(vix_data, tickers=['AAPL', 'MSFT', 'GOOGL', 'JNJ', 'PG', 'KO', 'PEP', 'XOM']):
    print("\n--- Strategy 3: Asymmetric Alpha (Covered Call) ---")
    results = []
    start_date = (datetime.now() - timedelta(days=365*2)).strftime("%Y-%m-%d") # 2 Years for GARCH
    end_date = datetime.now().strftime("%Y-%m-%d")
    
    for ticker_symbol in tickers:
        hist = fetch_data(ticker_symbol, start_date, end_date)
        hist.index = hist.index.tz_localize(None)
        dates = hist.index[::21]
        
        # Only simulate last 12 months
        dates = dates[dates > (datetime.now() - timedelta(days=365))]
        
        for i in range(len(dates)-1):
            entry_date = dates[i]
            exit_date = dates[i+1]
            
            spot_entry = hist.loc[entry_date]['Close']
            strike = spot_entry * 1.10 # 10% OTM
            
            # IV: GARCH + VIX
            hist_entry = hist.loc[:entry_date]
            garch_vol = get_garch_volatility(hist_entry)
            
            current_vix = 20.0
            if vix_data is not None:
                try: current_vix = vix_data.asof(entry_date)
                except: pass
            vix_scalar = max(1.0, current_vix / 20.0)
            
            atm_vol = garch_vol * vix_scalar
            iv_entry = get_vol_skew(atm_vol, strike, spot_entry)
            
            dte = 30 / 365.0
            mid_price = binomial_tree_price(spot_entry, strike, dte, RISK_FREE_RATE, iv_entry, option_type='call')
            _, sell_price = apply_transaction_costs(mid_price) # Selling the call
            
            spot_exit = hist.loc[exit_date]['Close']
            
            call_value_exit = max(0, spot_exit - strike)
            cost_to_close = call_value_exit
            if call_value_exit > 0:
                cost_to_close += (COMMISSION_PER_CONTRACT / 100)
            
            option_pnl = sell_price - cost_to_close
            total_pnl = (spot_exit - spot_entry + option_pnl) / spot_entry
            
            results.append({
                "Strategy": "Covered Call",
                "Ticker": ticker_symbol,
                "Period": f"{entry_date.date()}",
                "Return": total_pnl
            })
            
    avg_ret = np.mean([r['Return'] for r in results])
    print(f"Average Monthly Return: {avg_ret:.2%}")
    return results


In [23]:
results = []

res1 = run_analyst_day_strategy(vix_data)
if res1: results.append(res1)
results.extend(run_earnings_strategy(vix_data))
results.extend(run_covered_call_strategy(vix_data))

print("\n" + "="*50)
print("GOLDMAN SACHS STRATEGIES: BACKTEST REPORT v3.0")
print("="*50)

df_res = pd.DataFrame(results)
summary = df_res.groupby("Strategy")['Return'].agg(['count', 'mean', 'min', 'max'])
summary.columns = ['Trades', 'Avg Return', 'Min Return', 'Max Return']
summary['Win Rate'] = df_res.groupby("Strategy")['Return'].apply(lambda x: (x > 0).mean())

print("\n## 1. Performance Summary")
print(summary.to_markdown(floatfmt=".2%"))

print("\n## 2. Trade Log")
print(df_res[['Strategy', 'Ticker', 'Return']].to_markdown(index=False, floatfmt=".2%"))

with open("backtest_report_v3.md", "w") as f:
    f.write("# Goldman Sachs Strategies Backtest Report v3.0 (GARCH+VIX)\n\n")
    f.write("## 1. Performance Summary\n")
    f.write(summary.to_markdown(floatfmt=".2%"))
    f.write("\n\n## 2. Trade Log\n")
    f.write(df_res[['Strategy', 'Ticker', 'Return']].to_markdown(index=False, floatfmt=".2%"))
print("\n[+] Report saved to 'backtest_report_v3.md'")

Fetching VIX data...

--- Strategy 2: Pre-Earnings Call Buying ---


BLK Earnings 2025-10-14: 102.72%
BLK Earnings 2025-07-15: -89.55%
MSFT Earnings 2025-10-29: -63.58%
MSFT Earnings 2025-07-30: 98.12%
NVDA Earnings 2025-11-19: -87.39%
NVDA Earnings 2025-08-27: -42.56%
AAPL Earnings 2025-10-30: 1.72%
AAPL Earnings 2025-07-31: -100.34%
AMD Earnings 2025-11-04: -55.44%
AMD Earnings 2025-08-05: -89.44%
TSLA Earnings 2025-10-22: -34.79%
TSLA Earnings 2025-07-23: -87.48%
META Earnings 2025-10-29: -100.02%
META Earnings 2025-07-30: 374.48%
AMZN Earnings 2025-10-30: 199.61%
AMZN Earnings 2025-07-31: -100.10%

--- Strategy 3: Asymmetric Alpha (Covered Call) ---
Average Monthly Return: 1.43%

GOLDMAN SACHS STRATEGIES: BACKTEST REPORT v3.0

## 1. Performance Summary
| Strategy     |   Trades |   Avg Return |   Min Return |   Max Return |   Win Rate |
|:-------------|---------:|-------------:|-------------:|-------------:|-----------:|
| Covered Call | 8800.00% |        1.43% |       -9.92% |       12.55% |     56.82% |
| Earnings     | 1600.00% |       -4.63% |  